In [1]:
import urllib as u
from urllib.request import urlopen
import json
import pandas as pd
from pandas_datareader import data
import operator
import numpy as np
import re
from tqdm import tqdm

api_key='64d94994afb9689c49b660ebe9dee39c'

In [2]:
# Function to read in json data from FMP
def get_json_data(url):
    '''
    Scrape data (which must be json format) from given url
    Input: url to financialmodelingprep API
    Output: json file
    '''
    response = urlopen(url)
    dat = response.read().decode('utf-8')
    return json.loads(dat)

In [3]:
# Function to get sector of a Stock from its profile json file
def find_sector(json):
    
    for k, v in a.items():
        if k == 'symbol':
            symbol=v

    for k, v in a.items():
        if k == 'profile':
            profile=v

    for k, v in profile.items():
        if k == 'sector':
            sector=v
    return(symbol, sector)

In [4]:
# Function to get Year 2019 Financial Indicators from json file
def find_fin_indicators(json, indicator):
    cat = ['financials','growth','metrics']
    
    for k, v in json.items():
        if k == 'symbol':
            symbol=v

    for k, v in json.items():
        if k in cat:
            category = v

    indicators_2019 = [[(k,v) for k,v in d.items()] for d in category if d['date'][:4] == '2019'] # Generator creates a list of lists
    indicators_2019_list = [item for sublist in indicators_2019 for item in sublist] # Creating a flat list

    a = []

    for tup in (x for x in indicators_2019_list if x[0] in indicator):
        a.append((symbol,) + tup)
    return a

In [5]:
# Function to get Year 2019 Ratios from json file
def find_ratios_indicators(json, indicator):
    global ratios, symbol
    
    for k in json.items():
        symbol = json['symbol']
    
    for k in json.items():
        ratios = json['ratios']
            
    
    ratios_2019 = [d for d in ratios if d['date'][:4] == '2019']
    
    unpack1 = ratios_2019[0]['investmentValuationRatios']
    unpack2 = ratios_2019[0]['profitabilityIndicatorRatios']
    unpack3 = ratios_2019[0]['operatingPerformanceRatios']
    unpack4 = ratios_2019[0]['liquidityMeasurementRatios']
    unpack5 = ratios_2019[0]['debtRatios']
    unpack6 = ratios_2019[0]['cashFlowIndicatorRatios']
    
    unpack = unpack1
    unpack.update(unpack2)
    unpack.update(unpack3)
    unpack.update(unpack4)
    unpack.update(unpack5)
    unpack.update(unpack6)
    
    full_set = []
    reduced_set =[]

    for row in unpack.items():
            full_set.append(row)
    
    for tup in (x for x in full_set if x[0] in indicator):
        reduced_set.append((symbol,) +tup)
    
    return reduced_set

In [6]:
# Function to get the stock's price variation during the year 2020
def get_price_var(symbol):

    prices = data.DataReader(symbol, 'yahoo', '2020-01-01', '2020-12-31')['Adj Close']

    end = prices.index[-1]
    start = prices.index[0]

    price_var = ((prices[end] / prices[start]) - 1) * 100
    return symbol, price_var

In [7]:
# Leverage on Regex functionalities to retrieve FMP urls
with open('urls.txt') as f:
    urls = f.readlines()

'''List of Stocks'''
for i in urls:
    if len(re.findall('https.+list',i)) == 0:
        continue
    else: 
        list_of_stocks = str(re.findall('https.+list',i)[0])

'''Profiles'''
for i in urls:
    if len(re.findall('https.+profile\W',i)) == 0:
        continue
    else: 
        profiles = str(re.findall('https.+profile\W',i)[0])
        
'''Income Statement'''
for i in urls:
    if len(re.findall('https.+income.\w+\W',i)) == 0:
        continue
    else: 
        inc_stat = str(re.findall('https.+income.\w+\W',i)[0])
        
'''Cash Flow Statement'''
for i in urls:
    if len(re.findall('https.+cash-flow.\w+\W',i)) == 0:
        continue
    else: 
        cf_stat = str(re.findall('https.+cash-flow.\w+\W',i)[0])
        
'''Balance Sheet'''
for i in urls:
    if len(re.findall('https.+balance-sheet.\w+\W',i)) == 0:
        continue
    else: 
        bs_stat = str(re.findall('https.+balance-sheet.\w+\W',i)[0])
        
'''Financial Statment Growth'''
for i in urls:
    if len(re.findall('https.+growth\W',i)) == 0:
        continue
    else: 
        fs_growth = str(re.findall('https.+growth\W',i)[0])
        
'''Key Metrics'''
for i in urls:
    if len(re.findall('https.+metrics\W',i)) == 0:
        continue
    else: 
        key_metrics = str(re.findall('https.+metrics\W',i)[0])
        
'''Financial Ratios'''
for i in urls:
    if len(re.findall('https.+ratios\W',i)) == 0:
        continue
    else: 
        fin_ratios = str(re.findall('https.+ratios\W',i)[0])

In [8]:
# Full list of available stocks from FMP
url = list_of_stocks + '?apikey=' + api_key
ticks_json = get_json_data(url)

stocks = pd.DataFrame(ticks_json)
stocks = stocks[['symbol','name','exchange']]
stocks = stocks.dropna(how='any',subset=['symbol','name','exchange'])

In [9]:
# Limit the analysis to stocks of the New York Stock Exchange and Nasdaq Global
stocks_nyse = stocks[(stocks['exchange'] == 'Nasdaq Global Select') | (stocks['exchange'] == 'New York Stock Exchange')]

# For Testing purposes - To be dropped
stocks_nyse = stocks[(stocks['symbol'] == 'AAPL') | (stocks['symbol'] == 'WMT') | (stocks['symbol'] == 'ZION') | (stocks['symbol'] == 'SYNH') | (stocks['symbol'] == 'CI')]

stocks_nyse.reset_index(drop=True, inplace=True)
tickers_nyse = stocks_nyse['symbol'].tolist()

In [10]:
# Get sectors from company profiles
tickers_nyse_sector = []

for tick in tqdm(tickers_nyse):
    url = profiles + tick + '?apikey=' + api_key 
    a = get_json_data(url)
    tickers_nyse_sector.append(find_sector(a))

100%|██████████| 5/5 [00:09<00:00,  1.91s/it]


In [11]:
tickers_nyse_sector = pd.DataFrame(tickers_nyse_sector)
tickers_nyse_sector.columns = ['symbol', 'sector']

stocks_nyse = stocks_nyse.merge(tickers_nyse_sector)

In [12]:
# Load financial indicators from the provided .txt file
indicators = []
filename = 'indicators.txt'
with open(filename, 'r') as f:
    for line in f:
        indicators.append(line.strip('\n'))

In [13]:
# Get all annual indicators
inc_statement_indicators = []
cash_flow_indicators = []
bs_indicators = []
fs_growth_indicators = []
metrics_indicators = []
ratios_indicators = []

for tick in tqdm(tickers_nyse):
    try:
        url0 = inc_stat + tick + '?limit=120&apikey=' + api_key
        url1 = cf_stat + tick + '?limit=120&apikey=' + api_key
        url2 = bs_stat + tick + '?limit=120&apikey=' + api_key
        url3 = fs_growth + tick + '?limit=120&apikey=' + api_key
        url4 = key_metrics + tick + '?limit=40&apikey=' + api_key
        url5 = fin_ratios + tick + '?limit=40&apikey=' + api_key
        
        a0 = get_json_data(url0)
        a1 = get_json_data(url1)
        a2 = get_json_data(url2)
        a3 = get_json_data(url3)
        a4 = get_json_data(url4)
        a5 = get_json_data(url5)
    except:
        pass
    
    try:
        inc_statement_indicators.append(find_fin_indicators(a0,indicators))
        cash_flow_indicators.append(find_fin_indicators(a1,indicators))
        bs_indicators.append(find_fin_indicators(a2,indicators))
        fs_growth_indicators.append(find_fin_indicators(a3,indicators))
        metrics_indicators.append(find_fin_indicators(a4,indicators))
        ratios_indicators.append(find_ratios_indicators(a5,indicators))
    except:
        pass
    
    fin_indicators = inc_statement_indicators + cash_flow_indicators + bs_indicators + fs_growth_indicators + metrics_indicators + ratios_indicators
    fin_indicators_list = [item for sublist in fin_indicators for item in sublist]
    fin_indicators_list.sort(key = operator.itemgetter(0))

100%|██████████| 5/5 [00:55<00:00, 11.10s/it]


In [14]:
# Transform the list of indicators in a DataFrame
df_ind = pd.DataFrame(indicators)
df_ind.columns = ['variable']

In [58]:
# Dataframe by symbol whose indicators were found 
temp = []
df = df_ind

tickers_found = []

for j in fin_indicators_list:
    tickers_found.append(j[0])
    
tickers_found = pd.DataFrame(tickers_found)
tickers_found.columns = ['symbol']
tickers_found = tickers_found.drop_duplicates()
tickers_found.set_index('symbol')
tickers_found = tickers_found['symbol'].to_list()

for i in tqdm(tickers_found):
    for j in fin_indicators_list:
        if j[0] == i:
            temp.append(j[2])
            df_temp = pd.DataFrame(temp)
            df_temp.columns = [j[0]]
    df = df.join(df_temp)
    del df_temp
    temp.clear()

100%|██████████| 5/5 [00:00<00:00, 15.63it/s]


In [16]:
# Set Symbol as Index in Stocks List
stocks_nyse_labelled = stocks_nyse.set_index('symbol')

In [17]:
# Set symbol as Index in Dataframe with Indicators
df_labelled = df.set_index('variable')
df_labelled_trasposed = df_labelled.T
df_labelled_trasposed = df_labelled_trasposed.reset_index()
data_final = df_labelled_trasposed.rename(columns={'index': 'symbol'})
data_final = data_final.set_index('symbol')
data_final = data_final.merge(stocks_nyse_labelled['sector'], left_on=['symbol'], right_on=['symbol'])

In [18]:
# Get 2020 returns and save results in a DataFrame
returns = []
for i in tqdm(tickers_nyse):
    returns.append(get_price_var(i))
    
df_returns = pd.DataFrame(returns)
df_returns.columns = ['symbol','return']
df_returns = df_returns.set_index('symbol')

100%|██████████| 5/5 [00:08<00:00,  1.77s/it]


In [19]:
# DataFrame columns to numeric and merge with Returns
cols = [i for i in data_final.columns if i not in ['sector']]

for col in cols:
    data_final[col]=pd.to_numeric(data_final[col])
    
data_final = data_final.merge(df_returns, left_on=['symbol'], right_on=['symbol'])

In [20]:
data_final.head()

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,Earnings before Tax,Net Income,Preferred Dividends,Net Income Com,EPS,Dividend per Share,...,assetTurnover,currentRatio,quickRatio,cashRatio,debtRatio,debtEquityRatio,totalDebtToCapitalization,interestCoverage,sector,return
symbol,,,,,,,,,,,,,,,,,,,,,
AAPL,2.601740e+11,-0.020411,1.617820e+11,9.839200e+10,6.573700e+10,5.525600e+10,0.0,5.525600e+10,2.9925,3.037054,...,0.376708,-0.065129,0.037917,-0.074397,-0.093747,-0.056218,0.139154,0.092188,Technology,78.239958
CI,1.537430e+11,2.127975,0.000000e+00,0.000000e+00,6.570000e+09,5.104000e+09,0.0,5.104000e+09,13.5800,0.039902,...,0.000000,-0.010263,0.000000,0.016629,-0.274943,-0.194753,0.000000,0.177560,Healthcare,1.601726
SYNH,4.675815e+09,0.065078,3.645905e+09,1.029910e+09,1.017090e+08,1.312580e+08,0.0,1.312580e+08,1.2700,0.000000,...,0.000000,0.037327,0.000000,0.027414,0.058661,-0.064080,0.000000,0.098389,Healthcare,14.484959
WMT,5.144050e+11,0.028105,3.853010e+11,1.291040e+11,1.146000e+10,6.670000e+09,0.0,6.670000e+09,2.2800,2.071986,...,0.061512,0.119166,0.011100,0.072232,-0.048022,0.296644,0.000000,0.005981,Consumer Defensive,23.218719
ZION,2.834000e+09,NaN,0.000000e+00,0.000000e+00,1.053000e+09,8.160000e+08,NaN,8.160000e+08,4.4100,NaN,...,1.792527,0.000000,0.000000,0.006197,0.067376,-0.479518,0.000000,0.041071,Financial Services,-14.031578


In [31]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, AAPL to ZION
Data columns (total 72 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Revenue                         5 non-null      float64
 1   Revenue Growth                  4 non-null      float64
 2   Cost of Revenue                 5 non-null      float64
 3   Gross Profit                    5 non-null      float64
 4   Earnings before Tax             5 non-null      float64
 5   Net Income                      5 non-null      float64
 6   Preferred Dividends             4 non-null      float64
 7   Net Income Com                  5 non-null      float64
 8   EPS                             5 non-null      float64
 9   Dividend per Share              4 non-null      float64
 10  Gross Margin                    5 non-null      float64
 11  EBITDA Margin                   5 non-null      float64
 12  EBIT Margin                     4 non-n

In [26]:
for j in fin_indicators_list:
        if j[0]=='AAPL':
            print(j[1])

Revenue
Revenue Growth
Cost of Revenue
Gross Profit
Earnings before Tax
Net Income
Preferred Dividends
Net Income Com
EPS
Dividend per Share
Gross Margin
EBITDA Margin
EBIT Margin
Profit Margin
Free Cash Flow margin
EBITDA
EBIT
Consolidated Income
Earnings Before Tax Margin
Net Profit Margin
Stock-based compensation
Operating Cash Flow
Acquisitions and disposals
Investment purchases and sales
Investing Cash flow
Issuance (repayment) of debt
Issuance (buybacks) of shares
Dividend payments
Financing Cash Flow
Net cash flow / Change in cash
Free Cash Flow
Net Cash/Marketcap
Total assets
Total liabilities
Retained earnings (deficit)
Total shareholders equity
Other Assets
Other Liabilities
Gross Profit Growth
EBIT Growth
Operating Income Growth
Net Income Growth
EPS Growth
Weighted Average Shares Growth
Weighted Average Shares Diluted Growth
Dividends per Share Growth
Operating Cash Flow growth
Free Cash Flow growth
10Y Revenue Growth (per Share)
5Y Revenue Growth (per Share)
3Y Revenue Gro